# Clasificador Bayesiano - TAN

Se muestra un ejemplo de clasificación binaria con los datos de `german-credit`.

Importación de librerías y carga de datos

In [1]:
import os

import pandas as pd
import numpy as np

import pyAgrum.skbn as skbn

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

datos = pd.read_csv("../datos/credit_g.csv")

datos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 21 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   checking_status         1000 non-null   object
 1   duration                1000 non-null   int64 
 2   credit_history          1000 non-null   object
 3   purpose                 1000 non-null   object
 4   credit_amount           1000 non-null   int64 
 5   savings_status          1000 non-null   object
 6   employment              1000 non-null   object
 7   installment_commitment  1000 non-null   int64 
 8   personal_status         1000 non-null   object
 9   other_parties           1000 non-null   object
 10  residence_since         1000 non-null   int64 
 11  property_magnitude      1000 non-null   object
 12  age                     1000 non-null   int64 
 13  other_payment_plans     1000 non-null   object
 14  housing                 1000 non-null   object
 15  exist

Se hace una conversión a variables categóricas y se convierte a numérica (1, 0) la variable target:
- Si el préstamo es default, se convierte a 1. En caso, contrario es 0

In [2]:
# Pasamos las variables a categóricas
datos['checking_status'] = datos['checking_status'].astype('category')
datos['credit_history'] = datos['credit_history'].astype('category')
datos['purpose'] = datos['purpose'].astype('category')
datos['savings_status'] = datos['savings_status'].astype('category')
datos['employment'] = datos['employment'].astype('category')
datos['personal_status'] = datos['personal_status'].astype('category')
datos['other_parties'] = datos['other_parties'].astype('category')
datos['property_magnitude'] = datos['property_magnitude'].astype('category')
datos['other_payment_plans'] = datos['other_payment_plans'].astype('category')
datos['housing'] = datos['housing'].astype('category')
datos['job'] = datos['job'].astype('category')
datos['property_magnitude'] = datos['property_magnitude'].astype('category')
datos['own_telephone'] = datos['own_telephone'].astype('category')
datos['foreign_worker'] = datos['foreign_worker'].astype('category')
datos['class'] = datos['class'].astype('category')

# La variable class es una variable reservada en diferentes módulos de Python -> reemplazar por por target
datos.rename(columns={'class': 'target'}, inplace=True)
datos['target']=np.where(datos['target']=='good', 0, 1) # cambio en la codificación por sencillez en el preprocesado

Una vez esto, tiene lugar la definición de la muestra:
- Conversión a variables dummy de las variables categóricas (excluyendo un nivel para evitar multicolinealidad)
- Partición de la muestra (train y test)
- Estandarización de los datos (sobre la muestra de entrenamiento y aplicación de resultados sobre muestra de test)

In [3]:
# Definición de la muestra de trabajo
datos_entrada = datos.drop('target', axis=1) # Datos de entrada
datos_entrada = pd.get_dummies(datos_entrada, drop_first=True, dtype=int) #conversión a variables dummy

target = datos["target"] # muestra del target

In [4]:
# Partición de la muestra

test_size = 0.3 # muestra para el test 
seed = 222 # semilla

X_train, X_test, y_train, y_test = train_test_split(
    datos_entrada, target, test_size=test_size, random_state=seed, stratify=target
)

# Estandarización de la muestra
esc = StandardScaler().fit(X_train) # valores media y std de los datos de train

# aplicación a los datos de train y test
X_train_esc = esc.transform(X_train)
X_test_esc = esc.transform(X_test)

Creación y ajuste del clasificador TAN

In [5]:
# Creación del clasificador TAN en python
bayesian_network = skbn.BNClassifier(
    learningMethod='TAN',
    prior='Smoothing',
    scoringType='BIC',
    priorWeight=0.5,
    discretizationStrategy='quantile',
    usePR=True,
    significant_digit = 6
)

bayesian_network.fit(X_train, y_train) # ajuste del modelo

Predicciones en las muestras de train y test. Uso de accuracy como métrica de bondad de ajuste

In [8]:
# predicciones para la muestra de train y test

train_probs = bayesian_network.predict_proba(X_train)  
test_probs = bayesian_network.predict_proba(X_test)

# predict-proba proporciona las probabilidades

def preds_ones(probs, threshold = 0.5):
    return np.where(probs[:, 0] > threshold, 0, 1)

y_train_pred = preds_ones(train_probs)
y_test_pred = preds_ones(test_probs)

print(f'Accuracy (train) {round(accuracy_score(y_train, y_train_pred),2)}')
print(f'Accuracy (test) {round(accuracy_score(y_test, y_test_pred), 2)}')

Accuracy (train) 0.82
Accuracy (test) 0.73
